<a href="https://colab.research.google.com/github/alex-escobar-h/Facial-Expression-Recognition-Model/blob/main/facial_expression_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Facial Expression Recognition

## Install FER-2013 Dataset from Kaggle

In [3]:
from google.colab import userdata
import os
import torch
import torch.nn as nn
import torch.optim as optim

os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
!pip install -q kaggle
!kaggle datasets download -d msambare/fer2013
!unzip -q fer2013.zip

Dataset URL: https://www.kaggle.com/datasets/msambare/fer2013
License(s): DbCL-1.0
fer2013.zip: Skipping, found more recently modified local copy (use --force to force download)
replace test/angry/PrivateTest_10131363.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## Data Preparation and Transforms

In [8]:
import torch
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

normalize = transforms.Normalize(mean=[0.5],std=[0.5])
batch_size = 64
num_workers = 0

# Define transforms
train_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(48,scale=(0.9, 1.1), ratio=(0.9, 1.1)),
    transforms.ToTensor(),
    normalize
])

val_test_transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize((48,48)),
    transforms.ToTensor(),
    normalize
])

# Load datasets
train_dataset_full = datasets.ImageFolder(root='/content/train', transform=train_transform)

# Split training data into training and validation (80%/20%)
val_dataset_size = int(0.2*len(train_dataset_full))
train_dataset_size = len(train_dataset_full) - val_dataset_size

# Randomly split the full training dataset into 80% training and 20% validation subsets.
# The manual seed ensures reproducibility - the same split every time this code runs.
train_dataset, val_dataset = random_split(
    train_dataset_full,[train_dataset_size,val_dataset_size], generator=torch.Generator().manual_seed(42)
)

train_dataset.dataset.transform=train_transform
val_dataset.dataset.transform=val_test_transform

test_dataset = datasets.ImageFolder('/content/test', transform=val_test_transform)

# Wrap datasets in DataLoaders
train_loader = DataLoader(train_dataset,batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_loader = DataLoader(val_dataset,batch_size=batch_size, shuffle=False, num_workers=num_workers)
test_loader = DataLoader(test_dataset,batch_size=batch_size, shuffle=False, num_workers=num_workers)

# Retrieve classifications from training dataset
classes = train_dataset.dataset.classes
print(classes)

['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


## CNN Model Class

In [14]:
import torch.nn as nn
import torch.nn.functional as F


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(
            in_channels=20, out_channels=40, kernel_size=3, padding=1
        )
        self.conv3 = nn.Conv2d(
            in_channels=40, out_channels=80, kernel_size=3, padding=1
        )

        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout = nn.Dropout(0.25)

        # After 3 conv + 3 pool layers:
        # 48 -> 24 -> 12 -> 6 -> final feature map size: [80, 6, 6]
        self.fc1 = nn.Linear(in_features=80 * 6 * 6, out_features=600)
        self.fc2 = nn.Linear(in_features=600, out_features=7)  # 7 emotion classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [20, 24, 24]
        x = self.pool(F.relu(self.conv2(x)))  # [40, 12, 12]
        x = self.pool(F.relu(self.conv3(x)))  # [80, 6, 6]

        x = x.view(-1, 80 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

## Training and Validation Loop

In [15]:
import torch.optim as optim
import numpy as np

model = CNN()

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    model = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

n_epochs = 5
valid_loss_min = np.inf  # track change in validation loss

for epoch in range(1, n_epochs + 1):
    train_loss = 0.0
    valid_loss = 0.0

    ###################
    # Train the model #
    ###################
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * data.size(0)

    ######################
    # Validate the model #
    ######################
    model.eval()
    for batch_idx, (data, target) in enumerate(val_loader):
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()

        output = model(data)
        loss = criterion(output, target)
        valid_loss += loss.item() * data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    valid_loss = valid_loss / len(val_loader.dataset)

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))

    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            valid_loss_min, valid_loss))
        torch.save(model.state_dict(), 'model_trained.pt')
        valid_loss_min = valid_loss

CUDA is available!  Training on GPU ...
Epoch: 1 	Training Loss: 1.612053 	Validation Loss: 1.413038
Validation loss decreased (inf --> 1.413038).  Saving model ...
Epoch: 2 	Training Loss: 1.352848 	Validation Loss: 1.315712
Validation loss decreased (1.413038 --> 1.315712).  Saving model ...
Epoch: 3 	Training Loss: 1.222326 	Validation Loss: 1.231621
Validation loss decreased (1.315712 --> 1.231621).  Saving model ...
Epoch: 4 	Training Loss: 1.112960 	Validation Loss: 1.213738
Validation loss decreased (1.231621 --> 1.213738).  Saving model ...
Epoch: 5 	Training Loss: 1.006312 	Validation Loss: 1.225506


## Testing

In [11]:
model.eval()

test_loss = 0.0
num_classes = len(classes)
class_correct = [0. for _ in range(num_classes)]
class_total = [0. for _ in range(num_classes)]

with torch.no_grad():
    for data, target in test_loader:
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()

        output = model(data)
        loss = criterion(output, target)
        test_loss += loss.item() * data.size(0)

        _, pred = torch.max(output, 1)
        correct_tensor = pred.eq(target.view_as(pred))
        correct = correct_tensor.cpu().numpy() if train_on_gpu else correct_tensor.numpy()

        for i in range(len(target)):
            label = target[i].item()
            class_correct[label] += correct[i].item()
            class_total[label] += 1

# Compute average loss
test_loss = test_loss / len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

# Print per-class accuracy
for i in range(num_classes):
    if class_total[i] > 0:
        acc = 100 * class_correct[i] / class_total[i]
        print('Test Accuracy of %8s: %2d%% (%d/%d)' %
              (classes[i], int(acc), int(class_correct[i]), int(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no test examples)' % classes[i])

# Print overall accuracy
print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 1.111451

Test Accuracy of    angry: 56% (537/958)
Test Accuracy of  disgust: 40% (45/111)
Test Accuracy of     fear: 20% (215/1024)
Test Accuracy of    happy: 82% (1468/1774)
Test Accuracy of  neutral: 57% (708/1233)
Test Accuracy of      sad: 44% (559/1247)
Test Accuracy of surprise: 76% (638/831)

Test Accuracy (Overall): 58% (4170/7178)
